In [1]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, FastMarkerCluster

# Folium interactive map
  - [Multi-layer map tutorial (video)](https://www.youtube.com/watch?v=Ftczp3bx1uw)
  - [Multi-layer map tutorial (video)](https://codeburst.io/how-i-understood-displaying-interactive-maps-using-python-leaflet-js-and-folium-bd9b98c26e0e)
  - [Multi-layer map example (repo)](https://github.com/joeljogy/Display-interactive-leaflet-maps/blob/master/map.py)

In [2]:
# Jupyter struggles when maps have lots of elements on them
# Use the function below to deal with this issue
from IPython.display import IFrame

def embed_map(m, width='100%', height='750px', filepath='maps/', filename='map.html'):

    m.save(filepath+filename)
    return IFrame(filepath+filename, width=width, height=height)

## Load insurance policy data

In [3]:
policies = gpd.read_file('data/processed/policies.geojson')

policies.head()

,SampleNo,DriverGender,DriverAge,YearsLicensed,Licensed Age,WindscreenExtension,RentalCar,Alarm,GarageType,FinanceType,...,Start Date,Immobliser,HasPassengerFare,HasTradeUse,HCusinessUseType,IsACustomer,Vehicle Year,Make,Model,geometry
0,1,Female,49,31,18,No,No,No,Carport,NO,...,TODAY,No,No,No,No,No,2019,Foton,Sauvana,POINT (175.02788 -41.14489)
1,2,Male,28,9,19,No,No,No,Garage,NO,...,TODAY,No,No,No,No,No,2008,Fiat,Ducato,POINT (175.01538 -41.14356)
2,3,Male,27,9,18,No,No,No,Carport,NO,...,TODAY,No,No,No,No,No,2018,Holden,Astra,POINT (175.14368 -41.09620)
3,4,Male,52,34,18,No,No,No,Driveway,NO,...,TOMORROW,No,No,No,No,No,2010,Suzuki,Grand Vitara,POINT (175.10925 -41.09523)
4,5,Male,31,12,19,No,No,Yes,Driveway,NO,...,TODAY,No,No,No,No,No,2002,Ford,Falcon,POINT (175.07158 -41.11921)


In [4]:
# check that CRS is correct
policies.crs

{'init': 'epsg:4326'}

In [5]:
policies.shape

(3000, 35)

## 1. Marker Cluster
The `policies` dataframe contains 3000 individual policies, each with a lat/long coordinate pair. Plotting all 3000 would create a very cluttered map, so a MarkerCluster layer can be used to group policies when zoomed out.

Individual Marker objects can be be added to the MarkerCluster. Each Marker can contain attributes for popup and tooltip text, so that the user is presented with additional information when they hover or click.

In [6]:
# create basemap, centred on Auckland
m = folium.Map(
    location=[-36.848461, 174.763336],
    tiles='cartodbpositron',
    zoom_start=12
)

# add cluster marker to map
marker = MarkerCluster(name='Policies').add_to(m)

# add all policies to cluster marker
for idx, row in policies.iterrows():
    
    popup='Years Licensed:<br>' + str(row['YearsLicensed']) + '<br>Sum Insured:<br>' + str(row['Vehicle Sum Insured'])
    
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=popup,
        tooltip=row.SampleNo
    ).add_to(marker)
    
# allow this layer to be turned on and off
folium.LayerControl().add_to(m)

# show map
embed_map(m, width='75%', height='500px', filename='marker-cluster.html')

An alternative to the MarkerCluster is the FastMarkerCluster which can render much quicker, especially when working with a large number of points.

However, to add tooltips and popups will require custom JavaScript callbacks to be written. Find an example of this [here](https://stackoverflow.com/questions/50661316/adding-text-to-folium-fastmarkercluster-markers).

In [7]:
# create basemap, centred on Auckland
m = folium.Map(
    location=[-36.848461, 174.763336],
    tiles='cartodbpositron',
    zoom_start=12
)

# zip lat/long pairs into one list
pairs = list(zip(policies['geometry'].y.values.tolist(), policies['geometry'].x.values.tolist()))

# add cluster marker to map
marker = FastMarkerCluster(data=pairs, name='Policies').add_to(m)

# allow this layer to be turned on and off
folium.LayerControl().add_to(m)

# show map
embed_map(m, width='75%', height='500px', filename='marker-cluster.html')

## 2. GeoJSON overlays
  - [Folium quickstart guide](https://python-visualization.github.io/folium/quickstart.html#GeoJSON/TopoJSON-Overlays)
  
Polygon features can also be added to maps. The code below loads adds the ward boundaries directly from file to the map.

In [8]:
# create basemap, centred on Auckland
m = folium.Map(
    location=[-36.848461, 174.763336],
    tiles='cartodbpositron',
    zoom_start=12
)

# add ward polygons to map
folium.GeoJson(
    'data/processed/wards.geojson',
    name='wards'
).add_to(m)

# allow this layer to be turned on and off
folium.LayerControl().add_to(m)

embed_map(m, width='75%', height='500px', filename='geojson-overlay.html')

## 3. Choropleth maps
  - [Folium Quickstart guide](https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps)
  
Choropleth maps use colour to show attribute values of polygons. The code below shows the average sum insured of each ward. The colour palette and opacity can be customised using the `fill_color` and `fill_opacity` arguments.

In [9]:
# load metadata
df = gpd.read_file('data/processed/wards.geojson')

df.head(3)

,Ward_name,n_policies,avg_age,avg_n_claims,avg_sum_insured,avg_vehicle_year,Ward_no,geometry
0,Ahuriri Ward,4.0,34.250000,0.000000,32939.500000,2012.500000,06801,"MULTIPOLYGON (((169.64249 -44.58830, 169.64253..."
1,Ahuriri Ward,4.0,34.250000,0.000000,32939.500000,2012.500000,03101,"MULTIPOLYGON (((176.84970 -39.42650, 176.84966..."
2,Albany Ward,233.0,35.021459,0.330472,32538.197425,2014.094421,07602,"MULTIPOLYGON (((174.73881 -36.62435, 174.73881..."


In [10]:
bins = [0, 20000, 40000, 60000, 80000, 100000, 120000]

m = folium.Map(
    location=[-36.848461, 174.763336],
    zoom_start=9
)

# aditional tile layers
folium.TileLayer(tiles='CartoDB Positron', name="CartoDB").add_to(m)
folium.TileLayer(tiles='Stamen Toner', name="Stamen Toner").add_to(m)
folium.TileLayer(tiles='Stamen Terrain', name="Stamen Terrain").add_to(m)

# add sum insured layer
folium.Choropleth(
    geo_data=df,
    name='Sum Insured',
    data=df,
    columns=['Ward_name', 'avg_sum_insured'],
    key_on='feature.properties.Ward_name',
    bins=bins,
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Sum Insured ($)'
).add_to(m)


folium.LayerControl().add_to(m)

embed_map(m, width='75%', height='500px', filename='choropleth.html')

# 4. Multi-layer map
  - [Folium FeatureGroup example](https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/FeatureGroup.ipynb)
  
It is also possible to have multiple layers on a map. These layers can then be toggled on and off using the layer control. When using multiple choropleth layers, [this notebook](https://nbviewer.jupyter.org/gist/BibMartin/f153aa957ddc5fadc64929abdee9ff2e) should provide a good example for biding colour bars to layers.

The code below creates a map which contains both a MarkerCluster and a Choropleth layer.

In [11]:
# map
m = folium.Map(
    location=[-36.848461, 174.763336],
    tiles='cartodbpositron',
    zoom_start=10
)

# policies feature group
fg1 = folium.FeatureGroup(name='Policies')

# cluster marker
marker_cluster = MarkerCluster().add_to(fg1)

# add all policies to cluster marker
for idx, row in policies.iterrows():
    # popup text
    popup='Years Licensed:<br>' + str(row['YearsLicensed']) + '<br>Sum Insured:<br>' + str(row['Vehicle Sum Insured'])
    # add marker to cluster marker object
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=popup,
        tooltip=row.SampleNo
    ).add_to(marker_cluster)
    
marker_cluster.add_to(fg1)
fg1.add_to(m)

# add sum insured layer
folium.Choropleth(
    geo_data=df,
    name='Sum Insured',
    data=df,
    columns=['Ward_name', 'avg_sum_insured'],
    key_on='feature.properties.Ward_name',
    bins=bins,
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Sum Insured ($)'
).add_to(m)
    
# allow this layer to be turned on and off
folium.LayerControl().add_to(m)

embed_map(m, width='75%', height='500px', filename='multi-layer.html')